# BentoML Demo: Iris Classifier with custom web UI


**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install -q bentoml "scikit-learn>=0.23.2"

## Create BentoService for model serving

In [3]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):

    @api(input=DataframeInput(), batch=True)
    def predict(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Writing iris_classifier.py


[autoreload of six failed: Traceback (most recent call last):
  File "/Users/lavi/opt/anaconda3/envs/mlops/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/lavi/opt/anaconda3/envs/mlops/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/lavi/opt/anaconda3/envs/mlops/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/lavi/opt/anaconda3/envs/mlops/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 287, in update_class
    old_obj = getattr(old, key)
  File "/Users/lavi/opt/anaconda3/envs/mlops/lib/python3.7/site-packages/six.py", line 93, in __get__
    setattr(obj, self.name, result)  # Invokes __set__.
AttributeError: 'NoneType' object has no attribute 'cStringIO'
]


In [4]:
%%writefile main.py
from sklearn import svm
from sklearn import datasets

from iris_classifier import IrisClassifier

if __name__ == "__main__":
    # Load training data
    iris = datasets.load_iris()
    X, y = iris.data, iris.target

    # Model Training
    clf = svm.SVC(gamma='scale')
    clf.fit(X, y)

    # Create a iris classifier service instance
    iris_classifier_service = IrisClassifier()

    # Pack the newly trained model artifact
    iris_classifier_service.pack('model', clf)

    # Save the prediction service to disk for model serving
    saved_path = iris_classifier_service.save()

Writing main.py


In [5]:
!python main.py

[2021-06-12 03:58:43,153] INFO - BentoService bundle 'IrisClassifier:20210612035841_76D71A' saved to: /Users/lavi/bentoml/repository/IrisClassifier/20210612035841_76D71A


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [6]:
!bentoml serve IrisClassifier:latest

[2021-06-12 03:58:51,760] INFO - Getting latest version IrisClassifier:20210612035841_76D71A
[2021-06-12 03:58:51,793] INFO - Starting BentoML API proxy in development mode..
[2021-06-12 03:58:51,801] INFO - Starting BentoML API server in development mode..
[2021-06-12 03:58:52,770] INFO - Your system nofile limit is 4096, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
 * Serving Flask app 'IrisClassifier' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:50922/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2021 03:59:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 03:59:06] "GET /static_content/main.css HTTP/1.1" 200 -
127.0.0.

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
# !bentoml serveIrisiClassifier:latest --run-with-ngrok

At this point you can test out the Rest API server either by opening http://localhost:5000 in a new tab which will serve the swagger docs:

![alt text](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/swagger.png)


or by making a curl request in a another terminal window:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[1,2,1,2]]' \
localhost:5000/predict
```

## Adding Custom Web Static Content


In [7]:
!curl https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/static.tar.xz -o static.tar.xz
!tar --xz -xf static.tar.xz
!rm static.tar.xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  197k  100  197k    0     0   124k      0  0:00:01  0:00:01 --:--:--  124k


Here we have a very simple web ui as our static content that bento will serve.
Now we will edit our bento service to point to this static directory.

Add `@web_static_content('./static')` to `iris_classifier.py`

**Note**: The path can be both relative or absolute. 

In [8]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService, web_static_content
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
@web_static_content('./static')
class IrisClassifier(BentoService):

    @api(input=DataframeInput(), batch=True)
    def test(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Overwriting iris_classifier.py


In [9]:
!python main.py

[2021-06-12 04:00:59,826] WARNING - Importing from "bentoml.artifact.*" has been deprecated. Instead, use`bentoml.frameworks.*` and `bentoml.service.*`. e.g.:, `from bentoml.frameworks.sklearn import SklearnModelArtifact`, `from bentoml.service.artifacts import BentoServiceArtifact`, `from bentoml.service.artifacts.common import PickleArtifact`
[2021-06-12 04:00:59,830] WARNING - `auto_pip_dependencies` parameter in `@env` is being deprecated soon,use `infer_pip_packages` instead, e.g. `@env(infer_pip_packages=True)`
[2021-06-12 04:01:01,504] INFO - BentoService bundle 'IrisClassifier:20210612040100_AC97D6' saved to: /Users/lavi/bentoml/repository/IrisClassifier/20210612040100_AC97D6


In [10]:
!bentoml serve IrisClassifier:latest

[2021-06-12 04:01:06,357] INFO - Getting latest version IrisClassifier:20210612040100_AC97D6
[2021-06-12 04:01:06,390] INFO - Starting BentoML API proxy in development mode..
[2021-06-12 04:01:06,394] INFO - Starting BentoML API server in development mode..
[2021-06-12 04:01:06,520] INFO - Your system nofile limit is 4096, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
[2021-06-12 04:01:07,751] WARNING - Importing from "bentoml.artifact.*" has been deprecated. Instead, use`bentoml.frameworks.*` and `bentoml.service.*`. e.g.:, `from bentoml.frameworks.sklearn import SklearnModelArtifact`, `from bentoml.service.artifacts import BentoServiceArtifact`, `from bentoml.service.artifacts.common import PickleArtifact`
[2021

Now if you visit http://localhost:5000/, you should be served with a beautiful UI:

![Custom UI](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/webui.png)

It's still possible to access the swagger docs at `/docs`

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [12]:
!bentoml containerize IrisClassifier:latest

[2021-06-12 04:03:25,403] INFO - Getting latest version IrisClassifier:20210612040100_AC97D6
Found Bento: /Users/lavi/bentoml/repository/IrisClassifier/20210612040100_AC97D6
Containerizing IrisClassifier:20210612040100_AC97D6 with local YataiService and docker daemon from local environment\Build container image: irisclassifier:20210612040100_AC97D6


In [14]:
!docker run --rm -p 5000:5000 irisclassifier:20210612040100_AC97D6 

[2021-06-11 22:35:54,530] INFO - Starting BentoML proxy in production mode..
[2021-06-11 22:35:54,535] INFO - Starting BentoML API server in production mode..
[2021-06-11 22:35:54 +0000] [20] [INFO] Starting gunicorn 20.1.0
[2021-06-11 22:35:55 +0000] [20] [INFO] Listening at: http://0.0.0.0:32855 (20)
[2021-06-11 22:35:55 +0000] [20] [INFO] Using worker: sync
[2021-06-11 22:35:55 +0000] [22] [INFO] Booting worker with pid: 22
[2021-06-11 22:35:55,010] INFO - Running micro batch service on :5000
[2021-06-11 22:35:55 +0000] [1] [INFO] Starting gunicorn 20.1.0
[2021-06-11 22:35:55 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2021-06-11 22:35:55 +0000] [1] [INFO] Using worker: aiohttp.worker.GunicornWebWorker
[2021-06-11 22:35:55 +0000] [24] [INFO] Booting worker with pid: 24
[2021-06-11 22:35:55,155] WARNING - Saved BentoService Python version mismatch: loading BentoService bundle created with Python version 3.7.10, but current environment version is 3.7.6.
[2021-06-11 22:35:

## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [15]:
!bentoml run IrisClassifier:latest predict --input '[[5, 4, 3, 2]]'  # doesn't run on M1 

[2021-06-12 04:10:17,947] INFO - Getting latest version IrisClassifier:20210612040100_AC97D6
[2021-06-12 04:10:19,373] WARNING - Importing from "bentoml.artifact.*" has been deprecated. Instead, use`bentoml.frameworks.*` and `bentoml.service.*`. e.g.:, `from bentoml.frameworks.sklearn import SklearnModelArtifact`, `from bentoml.service.artifacts import BentoServiceArtifact`, `from bentoml.service.artifacts.common import PickleArtifact`
[2021-06-12 04:10:19,382] WARNING - `auto_pip_dependencies` parameter in `@env` is being deprecated soon,use `infer_pip_packages` instead, e.g. `@env(infer_pip_packages=True)`
Error: bentoml-cli run failed: Can't find API 'predict' in service 'IrisClassifier'


## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [17]:
from bentoml import load

service = load(saved_path)

print(service.predict([[5,4,3,2]]))

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

